**Bài toán 1: Tìm tập phổ biến trong dữ liệu giao dịch mua hàng tại siêu thị.**  Dữ liệu có thể lấy từ 
https://www.kaggle.com/code/mgmarques/customer-segmentation-and-market-basket-analysis

**Nhiệm vụ 1:** sử dụng thư viện mlxtend để tìm tập phổ biến có trong dữ liệu 

1. Cài đặt các thư viện **mlxtend** và import các gói dữ liệu

In [2]:
# To install mlxtend, go to the Anaconda prompt and execute pip 
import matplotlib.pyplot as plt 
import mlxtend.frequent_patterns 
import mlxtend.preprocessing 
import numpy 
import pandas

2.Thực hiện một số tính toán để hiểu các khái niệm cơ bản như support, confident và lift

In [29]:
# danh sách các giao dịch mua hàng 
example = [ 
['milk', 'bread', 'apples', 'cereal', 'jelly', 'cookies', 'salad', 'tomatoes'], 
['beer', 'milk', 'chips', 'salsa', 'grapes', 'wine', 'potatoes', 'eggs', 'carrots'], 
['diapers', 'baby formula', 'milk', 'bread', 'chicken', 'asparagus', 'cookies'], 
['milk', 'cookies', 'chicken', 'asparagus', 'broccoli', 'cereal', 'orange juice'], 
['steak', 'asparagus', 'broccoli', 'chips', 'salsa', 'ketchup', 'potatoes', 'salad'], 
['beer', 'salsa', 'asparagus', 'wine', 'cheese', 'crackers', 'strawberries', 'cookies'], 
['chocolate cake', 'strawberries', 'wine', 'cheese', 'beer', 'milk', 'orange juice'], 
['chicken', 'peas', 'broccoli', 'milk', 'bread', 'eggs', 'potatoes', 'ketchup', 'crackers'], 
['eggs', 'bread', 'cheese', 'turkey', 'salad', 'tomatoes', 'wine', 'steak', 'carrots'], 
['bread', 'milk', 'tomatoes', 'cereal', 'chicken', 'turkey', 'chips', 'salsa', 'diapers'] 
] 
# Tính support 
# the number of transactions 
N = len(example) 
# the frequency of milk 
f_x = sum(['milk' in i for i in example]) 
# the frequency of bread 
f_y = sum(['bread' in i for i in example]) 
# the frequency of milk and bread 
f_x_y = sum([ 
 all(w in i for w in ['milk', 'bread']) 
 for i in example 
]) 
# support (supp) 
support = f_x_y / N 
print("Support = {}".format(round(support, 4))) # support = 0.4 
# confidence: x -> y 
confidence = support / (f_x / N) 
print("Confidence = {}".format(round(confidence, 4)))# Confidence = 0.5714 
# lift: x -> y 
lift = confidence / (f_y / N) 
print("Lift = {}".format(round(lift, 4))) # Lift = 1.1429 
# leverage: x -> y 
leverage = support - ((f_x / N) * (f_y / N)) 
print("Leverage = {}".format(round(leverage, 4))) 
# conviction: x -> y 
conviction = (1 - (f_y / N)) / (1 - confidence) 
print("Conviction = {}".format(round(conviction, 4)))

Support = 0.4
Confidence = 0.5714
Lift = 1.1429
Leverage = 0.05
Conviction = 1.1667


3.Nạp dữ liệu, làm sạch dữ liệu và biến đổi dữ liệu về định dạng phù hợp để thực hiện xây dựng mô hình
(chuyển dữ liệu từ 1 hóa đơn có nhiều dòng thành 1 hóa đơn chỉ có 1 dòng chứa các sản phẩm) 

In [31]:
import pandas as pd

online = pd.read_csv(
    "Dataset/Online Retail.csv",
    sep=";",
    encoding="latin1",
    on_bad_lines="skip",
    engine="python"
)
online.head()

# create new column called IsCPresent 
online['IsCPresent'] = ( # looking for C in InvoiceNo column 
    online['InvoiceNo'] # convert column to string type for the apply function below 
        .astype(str) # set element to 1 if C present otherwise 0 
        .apply(lambda x: 1 if x.find('C') != -1 else 0) 
) 
online1 = (
  online # filter out non-positive quantity values 
    .loc[online["Quantity"] > 0] # remove InvoiceNos starting with C 
    .loc[online['IsCPresent'] != 1] # column filtering 
    .loc[:, ["InvoiceNo", "Description"]]# dropping all rows with at least  one missing value 
 .dropna() 
) 
# extract unique invoice numbers as list 
invoice_no_list = online1.InvoiceNo.tolist() 
invoice_no_list = list(set(invoice_no_list)) 
print("Length of list of invoice numbers:  {ln}".format(ln=len(invoice_no_list))) 
# get 5000 transaction 
subset_invoice_no_list = invoice_no_list[0:5000] 
# filter data set down to based on subset of invoice number list 
online1 = online1.loc[online1["InvoiceNo"].isin(subset_invoice_no_list)] 
invoice_item_list = [] 
for num in list(set(online1.InvoiceNo.tolist())): 
 # filter data set down to one invoice number 
 tmp_df = online1.loc[online1['InvoiceNo'] == num] 
 # extract item descriptions and convert to list 
 tmp_items = tmp_df.Description.tolist() 
 # append list invoice_item_list 
 invoice_item_list.append(tmp_items) 
print(invoice_item_list[1:5]) 

Length of list of invoice numbers:  20136
[['RED ROCKING HORSE HAND PAINTED', 'ETCHED GLASS COASTER', "CLOCK MAGNET MUM'S KITCHEN", 'WOODLAND CHARLOTTE BAG', 'CHARLOTTE BAG DOLLY GIRL DESIGN', 'CHARLOTTE BAG SUKI DESIGN', 'RED COAT RACK PARIS FASHION', 'YELLOW COAT RACK PARIS FASHION', 'BLUE COAT RACK PARIS FASHION', 'REGENCY TEA PLATE ROSES ', 'PACK OF 6 BIRDY GIFT TAGS', 'FOLDING BUTTERFLY MIRROR RED  ', 'FOLDING BUTTERFLY MIRROR HOT PINK ', 'FOLDING BUTTERFLY MIRROR IVORY ', 'SPACEBOY LUNCH BOX ', 'DOLLY GIRL LUNCH BOX', 'REGENCY CAKESTAND 3 TIER', 'BLUE DINER WALL CLOCK', 'GINGHAM HEART DECORATION', 'TRAVEL SEWING KIT', 'SPACEBOY CHILDRENS CUP', 'SPACEBOY CHILDRENS BOWL', 'DOLLY GIRL CHILDRENS CUP', 'DOLLY GIRL CHILDRENS BOWL', 'DOILEY STORAGE TIN', 'SET OF 4 KNICK KNACK TINS DOILEY ', 'WHITE ROCKING HORSE HAND PAINTED'], ['STORAGE TIN VINTAGE DOILY ', ' TRELLIS COAT RACK'], ['PLEASE ONE PERSON METAL SIGN', 'SCOTTIE DOG HOT WATER BOTTLE', 'GREY HEART HOT WATER BOTTLE', 'DOG BOWL CH

4.Mã hóa dữ liệu về dạng dữ liệu mà giải thuật apriori trong gói mlxtend yêu cầu

In [33]:

online_encoder = mlxtend.preprocessing.TransactionEncoder() 
online_encoder_array = online_encoder.fit_transform(invoice_item_list) 
online_encoder_df = pandas.DataFrame( 
    online_encoder_array,  
    columns=online_encoder.columns_ 
) 
# this is a very big table, so for more easy viewing only a subset is printed 
online_encoder_df.loc[ 
 4970:4979,  
 online_encoder_df.columns.tolist()[0:8] 
] 
print("Data dimension (row count, col count):  {dim}".format(dim=online_encoder_df.shape)) 

Data dimension (row count, col count):  (5000, 3729)


5.Tìm tập phổ biến bằng giải thuật apriori

In [34]:
# case 1: default minimum support = 0.5 does not use colnames (item names) 
mod = mlxtend.frequent_patterns.apriori(online_encoder_df) 
# case 2: 
mod_minsupport = mlxtend.frequent_patterns.apriori( 
 online_encoder_df, 
 min_support=0.01 
) 
mod_minsupport.loc[0:6] 
# case 3: add colnames for easier interpretability 
mod_colnames_minsupport = mlxtend.frequent_patterns.apriori( 
 online_encoder_df, 
 min_support=0.01, 
 use_colnames=True 
) 
mod_colnames_minsupport['length'] = ( 
 mod_colnames_minsupport['itemsets'].apply(lambda x: len(x)) 
) 
mod_colnames_minsupport.loc[0:6] 

,support,itemsets,length
0,0.0110,( SET 2 TEA TOWELS I LOVE LONDON ),1
1,0.0156,(10 COLOUR SPACEBOY PEN),1
2,0.0136,(12 MESSAGE CARDS WITH ENVELOPES),1
3,0.0172,(12 PENCIL SMALL TUBE WOODLAND),1
4,0.0192,(12 PENCILS SMALL TUBE RED RETROSPOT),1
5,0.0182,(12 PENCILS SMALL TUBE SKULL),1
6,0.0142,(12 PENCILS TALL TUBE RED RETROSPOT),1
